# Load Data  

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!pip install plot-keras-history

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
import numpy as np
import pandas as pd
import random
import matplotlib.pyplot as plt
from tqdm import tqdm

from PIL import Image
from plot_keras_history import show_history, plot_history
import os
import glob

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

import seaborn as sns
from sklearn.metrics import classification_report, confusion_matrix
 
# deep learning libraries
import tensorflow as tf
from tensorflow import keras
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import applications
from keras.models import Sequential, load_model
from keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D, Flatten, Dense, Dropout
from keras.preprocessing import image

from tensorflow.keras import Sequential, layers, models
from tensorflow.keras import optimizers
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers.experimental.preprocessing import Rescaling

from tensorflow.keras.applications.efficientnet_v2 import EfficientNetV2L
#from tensorflow.keras.applications.vgg16 import VGG16

In [4]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


In [5]:
categories = ["dick1"  , "nodick1"]

In [6]:
%%time

data = []
target = []

for cat in categories:
    filelist = glob.glob('/content/drive/MyDrive/DataScience/aubergine/dataset/'+cat+ '/*.j*')
    target.extend(cat for _ in filelist)
    data.extend([np.array(Image.open(fname).resize((224,224))) for fname in filelist])



/usr/local/lib/python3.8/dist-packages/PIL/Image.py:2797: DecompressionBombWarning: Image size (108000000 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(


CPU times: user 1h 39min 50s, sys: 5min 44s, total: 1h 45min 35s
Wall time: 4h 39min 38s


In [8]:
#la commande bug il faut la lancé deux fois
for i in range(len(data)):
    if data[i].shape != (224,224,3):
        data.pop(i)
        target.pop(i)

In [9]:
data = np.stack(data,axis=0)

# Data Splitting

In [10]:
encoder = LabelEncoder()
target = encoder.fit_transform(target)

In [11]:
X_train , X_test , y_train , y_test = train_test_split(data,target , test_size=0.33)

data = 0

In [ ]:
X_test_norm = np.round((X_test/255),3)
X_train_norm = np.round((X_train/255),3)

In [ ]:
X_test =0
X_train=0

In [ ]:
X_train_norm.shape

In [ ]:
X_train_norm = X_train_norm.reshape(-1,224, 224,3)
X_test_norm = X_test_norm.reshape(-1,224,224,3)

In [ ]:
X_train_norm.shape

In [ ]:
y_train.shape

# Transfer Learning

## Building model

In [ ]:
X_train_norm[0].shape

In [ ]:
def load_model():
      
    model = EfficientNetV2L(weights="imagenet", include_top=False, input_shape=X_train_norm[0].shape)
    
    return model


In [ ]:
model = load_model()

In [ ]:
def set_nontrainable_layers(model):
    # Set the first layers to be untrainable
    model.trainable = False
    return model

In [ ]:
model = set_nontrainable_layers(model)

In [ ]:
def add_last_layers(model):
    '''Take a pre-trained model, set its parameters as non-trainable, and add additional trainable layers on top'''
    base_model = set_nontrainable_layers(model)
    flatten_layer = layers.Flatten()
    dense_layer_500 = layers.Dense(500, activation='relu')
    drop_out = layers.Dropout(0.5),
    dense_layer_256 = layers.Dense(256, activation='relu')
    prediction_layer = layers.Dense(1, activation='sigmoid')
    
    model = models.Sequential([
        base_model,
        flatten_layer,
        dense_layer_500,
        drop_out,
        dense_layer_256,
        drop_out,
        prediction_layer
    ])
    return model

In [ ]:
model = add_last_layers(model)

In [ ]:
def build_model():  
    model = load_model()
    model = add_last_layers(model)
    
    opt = optimizers.Adam(learning_rate=1e-4)
    model.compile(loss='binary_crossentropy',
                  optimizer=opt,
                  metrics=['accuracy'])
    return model

In [ ]:
model = build_model()

In [ ]:
model.summary()

In [ ]:
#from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.applications.efficientnet_v2 import preprocess_input

In [ ]:
X_train = preprocess_input(X_train_norm) 
X_test = preprocess_input(X_test_norm)

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping

model = build_model()

es = EarlyStopping(monitor = 'val_accuracy', 
                   mode = 'max', 
                   patience = 10, 
                   verbose = 1, 
                   restore_best_weights = True)

history = model.fit(X_train_norm, y_train,
                        validation_split=0.3,
                        epochs=500,
                        batch_size=32,
                        callbacks=[es])

In [ ]:
# Load the base models
'''efficientnetb7 = tf.keras.applications.efficientnet.EfficientNetB7(weights='imagenet', include_top=False, input_shape=(224,224,3))
efficientnetv2l = tf.keras.applications.efficientnet_v2.EfficientNetV2L(weights='imagenet', include_top=False, input_shape=(224,224,3))
convnextxlarge = tf.keras.applications.ConvNeXtXLarge(weights='imagenet', include_top=False, input_shape=(224,224,3))
resnet152v2 = tf.keras.applications.resnet_v2.ResNet152V2(weights='imagenet', include_top=False, input_shape=(224,224,3))
inceptionresnetv2 = tf.keras.applications.InceptionResNetV2(weights='imagenet', include_top=False, input_shape=(224,224,3))
nasnetlarge = tf.keras.applications.NASNetLarge(weights='imagenet', include_top=False, input_shape=(224,224,3))
vgg19 = tf.keras.applications.VGG19(weights='imagenet', include_top=False, input_shape=(224,224,3))

# Freeze the base models
efficientnetb7.trainable = False
efficientnetv2l.trainable = False
convnextxlarge.trainable = False
resnet152v2.trainable = False
inceptionresnetv2.trainable = False
nasnetlarge.trainable = False
vgg19.trainable = False

# Create the inputs
inputs = tf.keras.Input(shape=(224,224,3))

# Pass the inputs through the base models to obtain the features
x1 = efficientnetb7(inputs)
x1 = tf.keras.layers.GlobalAveragePooling2D()(x1)
x2 = efficientnetv2l(inputs)
x2 = tf.keras.layers.GlobalAveragePooling2D()(x2)
x3 = convnextxlarge(inputs)
x3 = tf.keras.layers.GlobalAveragePooling2D()(x3)
x4 = resnet152v2(inputs)
x4 = tf.keras.layers.GlobalAveragePooling2D()(x4)
x5 = inceptionresnetv2(inputs)
x5 = tf.keras.layers.GlobalAveragePooling2D()(x5)
x6 = nasnetlarge(inputs)
x6 = tf.keras.layers.GlobalAveragePooling2D()(x6)
x7 = vgg19(inputs)
x7 = tf.keras.layers.GlobalAveragePooling2D()(x7)


# Concatenate the features
concatenated = tf.keras.layers.concatenate([x1, x2, x3, x4, x5, x6, x7])

# Create a final model using the extracted features
model = tf.keras.Sequential([
    tf.keras.layers.Dense(1024, activation='relu', input_shape=concatenated.shape[1:]),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')])

# Compile the model with Stochastic Gradient Descent
optimizer = tf.keras.optimizers.Adam(learning_rate=0.01)
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy', 'precision'])

# Define the early stopping callback
early_stopping = tf.keras.callbacks.EarlyStopping(patience=10,
                                                   min_delta=0.001,
                                                   restore_best_weights=True)

# Define the batch size
batch_size = 32

history = model.fit(X_train_norm, y_train_oh,
                        validation_split=0.3,
                        epochs=500,
                        batch_size=batch_size,
                        callbacks=[early_stopping])'''

In [ ]:
plot_history(history)

## plot and evaluate

In [ ]:
predictions = model.predict(X_test_norm)

In [ ]:
predictions

In [ ]:
from sklearn.metrics import confusion_matrix

# Calculate the confusion matrix
cm = confusion_matrix(y_test, (predictions>0.5))

# Plot the confusion matrix
fig, ax = plt.subplots()
im = ax.imshow(cm, cmap='Blues')

# Add labels to the plot
ax.set_xticks(np.arange(2))
ax.set_yticks(np.arange(2))
ax.set_xticklabels(['0', '1'])
ax.set_yticklabels(['0', '1'])

# Loop over data dimensions and create text annotations
for i in range(2):
    for j in range(2):
        text = ax.text(j, i, cm[i, j], ha="center", va="center", color="black")

# Show plot
plt.show()

In [ ]:
fig = plt.figure(figsize=(20,15))
gs = fig.add_gridspec(6,6)

for line in range(0,6):
    for row in range(0,6):
        num_image = random.randint(0, X_test_norm.shape[0])
        ax = fig.add_subplot(gs[line,row])
        ax.axis('off');
        ax.set_title("Predicted : " + categories[list(np.round(predictions[num_image])).index(1)])
        ax.imshow(X_test_norm[num_image])
fig.suptitle("Predited label for the displayed picture",fontsize = 25 , x= 0.42)

In [ ]:
model.save("Model_dp")